In [ ]:
import pandas as pd
import numpy as np
from matplotlib import cm
from scipy.interpolate import griddata
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('logs/analyseParameters.csv')
df.head()

In [ ]:

# Setting up the plot style
sns.set(style="whitegrid")

# Creating subplots for each feature against `acc_sound`
fig, axs = plt.subplots(1, 3, figsize=(18, 6))  # 3 subplots in a row for `seq_len`, `gen_len`, `mem_len`

# Plotting `acc_sound` changes with `seq_len` grouped by `temp`
sns.lineplot(data=df, x='seq_len', y='acc_sound', hue='temp', marker='o', ax=axs[0], errorbar=None)
axs[0].set_title("acc_sound vs seq_len by temp")
axs[0].set_xlabel("seq_len")
axs[0].set_ylabel("acc_sound")

# Plotting `acc_sound` changes with `gen_len` grouped by `temp`
sns.lineplot(data=df, x='gen_len', y='acc_sound', hue='temp', marker='o', ax=axs[1], errorbar=None)
axs[1].set_title("acc_sound vs gen_len by temp")
axs[1].set_xlabel("gen_len")
axs[1].set_ylabel("acc_sound")

# Plotting `acc_sound` changes with `mem_len` grouped by `temp`
sns.lineplot(data=df, x='mem_len', y='acc_sound', hue='temp', marker='o', ax=axs[2], errorbar=None)
axs[2].set_title("acc_sound vs mem_len by temp")
axs[2].set_xlabel("mem_len")
axs[2].set_ylabel("acc_sound")

# Adjusting layout for clarity
plt.tight_layout()
plt.legend(title="temp", loc='upper right', bbox_to_anchor=(1.2, 1))
plt.show()

In [ ]:
# Step 1: Group by 'gen_len' and 'mem_len' and calculate the mean of 'acc_sound' for unique pairs
df_grouped = df.groupby(['gen_len', 'mem_len'], as_index=False)['acc_sound'].mean()

# Step 2: Define a high-resolution grid for smoother interpolation
gen_len_vals = np.linspace(df_grouped['gen_len'].min(), df_grouped['gen_len'].max(), 100)
mem_len_vals = np.linspace(df_grouped['mem_len'].min(), df_grouped['mem_len'].max(), 100)
X, Y = np.meshgrid(gen_len_vals, mem_len_vals)

# Step 3: Interpolate acc_sound values onto the high-resolution grid
Z = griddata(
    (df_grouped['gen_len'], df_grouped['mem_len']),
    df_grouped['acc_sound'],
    (X, Y),
    method='cubic'
)

# Step 4: Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plotting the surface with smoother color transitions
surf = ax.plot_surface(X, Y, Z, cmap=cm.viridis, edgecolor='none', rstride=1, cstride=1, antialiased=True)

# Adding color bar for reference
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

# Setting labels
ax.set_xlabel("gen_len")
ax.set_ylabel("mem_len")
ax.set_zlabel("acc_sound")
ax.set_title("3D Smooth Surface Plot of acc_sound vs gen_len and mem_len")

# Show plot
plt.show()